In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import xgboost as xgb
import pickle
import warnings
warnings.filterwarnings('ignore')

print("="*70)
print("CORN YIELD PREDICTION - ML TRAINING PIPELINE")
print("="*70)

CORN YIELD PREDICTION - ML TRAINING PIPELINE


In [3]:
# ============================================================================
# CONFIGURATION
# ============================================================================
RANDOM_STATE = 42
TEST_SIZE = 0.15
VAL_SIZE = 0.15
CV_FOLDS = 5

# ============================================================================
# 1. LOAD DATA
# ============================================================================
print("\n[1/8] Loading data...")

df = pd.read_csv('modeling_dataset_final.csv')
print(f"  ✓ Loaded {len(df):,} records")
print(f"  ✓ Features: {len(df.columns)}")


[1/8] Loading data...
  ✓ Loaded 82,436 records
  ✓ Features: 50


In [4]:
df.describe()

,Year,State ANSI,County ANSI,Yield_BU_ACRE,Area_Planted_ACRES,Area_Harvested_ACRES,Production_BU,Abandonment_Rate,Harvest_Efficiency,Soil_AWC,...,heat_moisture_stress,rh_mean,rh_reproductive,weeks_high_humidity,temp_early_vs_late,precip_early_vs_late,gdd_anomaly,precip_anomaly_mm,precip_anomaly_pct,temp_anomaly
count,82436.000000,82436.000000,82436.000000,82436.000000,82436.000000,82436.000000,8.243600e+04,82436.000000,82436.000000,82436.000000,...,82436.000000,82436.000000,82436.000000,82436.000000,82436.000000,82436.000000,82436.000000,82436.000000,82436.000000,82436.000000
mean,1999.957385,30.094631,95.828728,116.066207,40945.816997,37389.724877,5.281982e+06,0.183606,0.816394,0.146179,...,0.809306,69.666958,69.182528,5.445109,-5.568068,18.161273,-16.173655,-12.100269,-1.821947,-0.099322
std,12.173337,13.765798,76.271186,41.992524,52971.576973,51188.743041,8.282445e+06,0.214977,0.214977,0.049615,...,1.301745,9.298144,11.897232,5.367811,1.569149,139.300324,165.382218,127.328276,21.688039,0.916226
min,1981.000000,1.000000,1.000000,0.000000,10.000000,10.000000,4.500000e+02,0.000000,0.000532,0.000000,...,0.000000,21.677460,14.085357,0.000000,-13.082834,-820.350000,-1331.975333,-711.363333,-92.551390,-2.855399
25%,1989.000000,19.000000,39.000000,86.000000,4200.000000,2800.000000,2.620000e+05,0.031250,0.741554,0.115700,...,0.000000,65.932381,63.020982,1.000000,-6.540371,-67.342500,-134.223819,-93.482417,-16.362113,-0.761073
50%,1999.000000,29.000000,83.000000,114.200000,17000.000000,13400.000000,1.444300e+06,0.090909,0.909091,0.150000,...,0.376733,72.180899,71.905000,4.000000,-5.513599,15.645000,-14.790250,-18.564667,-3.362340,-0.096809
75%,2010.000000,42.000000,135.000000,145.600000,60900.000000,54800.000000,6.758400e+06,0.258446,0.968750,0.178600,...,1.178244,75.935668,78.303125,8.000000,-4.494381,100.122500,99.468556,57.966667,10.533614,0.547280
max,2023.000000,56.000000,810.000000,277.100000,397000.000000,394000.000000,7.722400e+07,0.999468,1.000000,0.540500,...,22.216051,86.296296,89.304286,52.000000,1.158148,1005.720000,1786.769389,1199.514667,223.797594,3.135909


In [5]:
# Create lag features for historical yield
df = df.sort_values(['State', 'County', 'Year'])
df['Yield_Lag1'] = df.groupby(['State', 'County'])['Yield_BU_ACRE'].shift(1)
df['Yield_Lag2'] = df.groupby(['State', 'County'])['Yield_BU_ACRE'].shift(2)
df['Yield_3yr_Avg'] = df.groupby(['State', 'County'])['Yield_BU_ACRE'].transform(
    lambda x: x.rolling(window=3, min_periods=1).mean().shift(1)
)

In [6]:
# Drop rows with NaN in lag features
df = df.dropna(subset=['Yield_Lag1', 'Yield_Lag2', 'Yield_3yr_Avg'])
print(f"  ✓ Created lag features")
print(f"  ✓ Records after feature engineering: {len(df):,}")

  ✓ Created lag features
  ✓ Records after feature engineering: 77,237


In [7]:
# Define feature columns
exclude_cols = [
    'Yield_BU_ACRE',  # Target
    'State', 'County',  # Identifiers (will encode)
    'Year',  # Already captured in features
    'State ANSI', 'County ANSI', 'Ag District',  # Redundant IDs
    'Area_Planted_ACRES', 'Area_Harvested_ACRES', 'Production_BU',  # Don't use for prediction
]

# Encode State as categorical
state_encoder = {state: idx for idx, state in enumerate(df['State'].unique())}
df['State_Encoded'] = df['State'].map(state_encoder)

# Select features
feature_cols = [col for col in df.columns if col not in exclude_cols]
print(f"  ✓ Selected {len(feature_cols)} features")

# Separate features and target
X = df[feature_cols]
y = df['Yield_BU_ACRE']

print(f"\n  Feature Groups:")
print(f"    Historical: Yield_Lag1, Yield_Lag2, Yield_3yr_Avg")
print(f"    Soil: Soil_AWC, Soil_Clay_Pct, Soil_pH, Soil_Organic_Matter_Pct")
print(f"    Weather: gdd_total, precip_total, temp_mean_season, etc. (34 features)")
print(f"    Other: Abandonment_Rate, Harvest_Efficiency, State_Encoded")

  ✓ Selected 44 features

  Feature Groups:
    Historical: Yield_Lag1, Yield_Lag2, Yield_3yr_Avg
    Soil: Soil_AWC, Soil_Clay_Pct, Soil_pH, Soil_Organic_Matter_Pct
    Weather: gdd_total, precip_total, temp_mean_season, etc. (34 features)
    Other: Abandonment_Rate, Harvest_Efficiency, State_Encoded


In [8]:
# ============================================================================
# 3. TRAIN/VAL/TEST SPLIT
# ============================================================================
print("\n[3/8] Creating train/validation/test splits...")

# First split: separate test set
X_temp, X_test, y_temp, y_test = train_test_split(
    X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE
)

# Second split: separate validation set from training
val_size_adjusted = VAL_SIZE / (1 - TEST_SIZE)
X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp, test_size=val_size_adjusted, random_state=RANDOM_STATE
)

print(f"  ✓ Training set:   {len(X_train):,} samples ({len(X_train)/len(X)*100:.1f}%)")
print(f"  ✓ Validation set: {len(X_val):,} samples ({len(X_val)/len(X)*100:.1f}%)")
print(f"  ✓ Test set:       {len(X_test):,} samples ({len(X_test)/len(X)*100:.1f}%)")


[3/8] Creating train/validation/test splits...
  ✓ Training set:   54,065 samples (70.0%)
  ✓ Validation set: 11,586 samples (15.0%)
  ✓ Test set:       11,586 samples (15.0%)


In [9]:
# ============================================================================
# 4. FEATURE SCALING
# ============================================================================
print("\n[4/8] Scaling features...")

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# Convert back to DataFrame for easier handling
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X_val_scaled = pd.DataFrame(X_val_scaled, columns=X_val.columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns)

print(f"  ✓ Features scaled (StandardScaler)")


[4/8] Scaling features...
  ✓ Features scaled (StandardScaler)


In [10]:
# ============================================================================
# 5. MODEL TRAINING & EVALUATION
# ============================================================================
print("\n[5/8] Training models...")
print("="*70)

models = {}
predictions = {}
metrics = {}

# ----------------------------------------------------------------------------
# MODEL 1: BASELINE - Historical Average
# ----------------------------------------------------------------------------
print("\n[Model 1/5] Baseline: Historical Average")
print("-" * 70)

# Use 3-year average as baseline prediction
baseline_pred_train = X_train['Yield_3yr_Avg']
baseline_pred_val = X_val['Yield_3yr_Avg']
baseline_pred_test = X_test['Yield_3yr_Avg']

# Metrics
baseline_metrics = {
    'train_mae': mean_absolute_error(y_train, baseline_pred_train),
    'train_rmse': np.sqrt(mean_squared_error(y_train, baseline_pred_train)),
    'train_r2': r2_score(y_train, baseline_pred_train),
    'val_mae': mean_absolute_error(y_val, baseline_pred_val),
    'val_rmse': np.sqrt(mean_squared_error(y_val, baseline_pred_val)),
    'val_r2': r2_score(y_val, baseline_pred_val),
    'test_mae': mean_absolute_error(y_test, baseline_pred_test),
    'test_rmse': np.sqrt(mean_squared_error(y_test, baseline_pred_test)),
    'test_r2': r2_score(y_test, baseline_pred_test)
}

print(f"  Training   - MAE: {baseline_metrics['train_mae']:.2f}, RMSE: {baseline_metrics['train_rmse']:.2f}, R²: {baseline_metrics['train_r2']:.3f}")
print(f"  Validation - MAE: {baseline_metrics['val_mae']:.2f}, RMSE: {baseline_metrics['val_rmse']:.2f}, R²: {baseline_metrics['val_r2']:.3f}")
print(f"  Test       - MAE: {baseline_metrics['test_mae']:.2f}, RMSE: {baseline_metrics['test_rmse']:.2f}, R²: {baseline_metrics['test_r2']:.3f}")

metrics['Baseline'] = baseline_metrics
predictions['Baseline'] = {'train': baseline_pred_train, 'val': baseline_pred_val, 'test': baseline_pred_test}


[5/8] Training models...

[Model 1/5] Baseline: Historical Average
----------------------------------------------------------------------
  Training   - MAE: 19.30, RMSE: 25.31, R²: 0.640
  Validation - MAE: 19.68, RMSE: 25.85, R²: 0.626
  Test       - MAE: 19.44, RMSE: 25.64, R²: 0.628


In [11]:
# ----------------------------------------------------------------------------
# MODEL 2: RIDGE REGRESSION
# ----------------------------------------------------------------------------
print("\n[Model 2/5] Ridge Regression with Hyperparameter Tuning")
print("-" * 70)

# Hyperparameter tuning
param_grid_ridge = {
    'alpha': [0.01, 0.1, 1.0, 10.0, 100.0]
}

ridge_model = Ridge(random_state=RANDOM_STATE)
grid_search_ridge = GridSearchCV(
    ridge_model, param_grid_ridge, cv=CV_FOLDS, 
    scoring='r2', n_jobs=-1, verbose=0
)
grid_search_ridge.fit(X_train_scaled, y_train)

print(f"  ✓ Best parameters: {grid_search_ridge.best_params_}")
print(f"  ✓ Best CV R²: {grid_search_ridge.best_score_:.3f}")

# Train final model
ridge_best = grid_search_ridge.best_estimator_
ridge_pred_train = ridge_best.predict(X_train_scaled)
ridge_pred_val = ridge_best.predict(X_val_scaled)
ridge_pred_test = ridge_best.predict(X_test_scaled)

ridge_metrics = {
    'train_mae': mean_absolute_error(y_train, ridge_pred_train),
    'train_rmse': np.sqrt(mean_squared_error(y_train, ridge_pred_train)),
    'train_r2': r2_score(y_train, ridge_pred_train),
    'val_mae': mean_absolute_error(y_val, ridge_pred_val),
    'val_rmse': np.sqrt(mean_squared_error(y_val, ridge_pred_val)),
    'val_r2': r2_score(y_val, ridge_pred_val),
    'test_mae': mean_absolute_error(y_test, ridge_pred_test),
    'test_rmse': np.sqrt(mean_squared_error(y_test, ridge_pred_test)),
    'test_r2': r2_score(y_test, ridge_pred_test)
}

print(f"  Training   - MAE: {ridge_metrics['train_mae']:.2f}, RMSE: {ridge_metrics['train_rmse']:.2f}, R²: {ridge_metrics['train_r2']:.3f}")
print(f"  Validation - MAE: {ridge_metrics['val_mae']:.2f}, RMSE: {ridge_metrics['val_rmse']:.2f}, R²: {ridge_metrics['val_r2']:.3f}")
print(f"  Test       - MAE: {ridge_metrics['test_mae']:.2f}, RMSE: {ridge_metrics['test_rmse']:.2f}, R²: {ridge_metrics['test_r2']:.3f}")

models['Ridge'] = ridge_best
metrics['Ridge'] = ridge_metrics
predictions['Ridge'] = {'train': ridge_pred_train, 'val': ridge_pred_val, 'test': ridge_pred_test}


[Model 2/5] Ridge Regression with Hyperparameter Tuning
----------------------------------------------------------------------


  ✓ Best parameters: {'alpha': 10.0}
  ✓ Best CV R²: 0.722
  Training   - MAE: 16.74, RMSE: 22.22, R²: 0.723
  Validation - MAE: 17.15, RMSE: 22.80, R²: 0.709
  Test       - MAE: 16.94, RMSE: 22.51, R²: 0.713


In [12]:
# ----------------------------------------------------------------------------
# MODEL 3: RANDOM FOREST (Randomized Search)
# ----------------------------------------------------------------------------
print("\n[Model 3/5] Random Forest with Hyperparameter Tuning (Randomized Search)")
print("-" * 70)

param_dist_rf = {
    'n_estimators': [100, 200, 300, 400],  # more options, random picks are cheap
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2']  # optional but often improves RF
}

rf_model = RandomForestRegressor(random_state=RANDOM_STATE, n_jobs=-1)

# 🔥 Randomized search instead of grid search
from sklearn.model_selection import RandomizedSearchCV

random_search_rf = RandomizedSearchCV(
    estimator=rf_model,
    param_distributions=param_dist_rf,
    n_iter=12,              # number of random combos to try (tune if needed)
    cv=CV_FOLDS,
    scoring='r2',
    n_jobs=-1,
    random_state=RANDOM_STATE,
    verbose=1
)

print("  Training Random Forest (much faster now)...")
random_search_rf.fit(X_train, y_train)

print(f"  ✓ Best parameters: {random_search_rf.best_params_}")
print(f"  ✓ Best CV R²: {random_search_rf.best_score_:.3f}")

rf_best = random_search_rf.best_estimator_

rf_pred_train = rf_best.predict(X_train)
rf_pred_val = rf_best.predict(X_val)
rf_pred_test = rf_best.predict(X_test)

rf_metrics = {
    'train_mae': mean_absolute_error(y_train, rf_pred_train),
    'train_rmse': np.sqrt(mean_squared_error(y_train, rf_pred_train)),
    'train_r2': r2_score(y_train, rf_pred_train),
    'val_mae': mean_absolute_error(y_val, rf_pred_val),
    'val_rmse': np.sqrt(mean_squared_error(y_val, rf_pred_val)),
    'val_r2': r2_score(y_val, rf_pred_val),
    'test_mae': mean_absolute_error(y_test, rf_pred_test),
    'test_rmse': np.sqrt(mean_squared_error(y_test, rf_pred_test)),
    'test_r2': r2_score(y_test, rf_pred_test)
}

print(f"  Training   - MAE: {rf_metrics['train_mae']:.2f}, RMSE: {rf_metrics['train_rmse']:.2f}, R²: {rf_metrics['train_r2']:.3f}")
print(f"  Validation - MAE: {rf_metrics['val_mae']:.2f}, RMSE: {rf_metrics['val_rmse']:.2f}, R²: {rf_metrics['val_r2']:.3f}")
print(f"  Test       - MAE: {rf_metrics['test_mae']:.2f}, RMSE: {rf_metrics['test_rmse']:.2f}, R²: {rf_metrics['test_r2']:.3f}")

models['RandomForest'] = rf_best
metrics['RandomForest'] = rf_metrics
predictions['RandomForest'] = {'train': rf_pred_train, 'val': rf_pred_val, 'test': rf_pred_test}


[Model 3/5] Random Forest with Hyperparameter Tuning (Randomized Search)
----------------------------------------------------------------------
  Training Random Forest (much faster now)...
Fitting 5 folds for each of 12 candidates, totalling 60 fits


Exception ignored in: <function ResourceTracker.__del__ at 0x105701da0>
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 82, in __del__
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 91, in _stop
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 116, in _stop_locked
ChildProcessError: [Errno 10] No child processes


  ✓ Best parameters: {'n_estimators': 300, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'max_depth': None}
  ✓ Best CV R²: 0.840
  Training   - MAE: 5.49, RMSE: 7.86, R²: 0.965
  Validation - MAE: 12.42, RMSE: 17.07, R²: 0.837
  Test       - MAE: 12.14, RMSE: 16.62, R²: 0.844


In [13]:
# ----------------------------------------------------------------------------
# MODEL 4: XGBOOST (Randomized Hyperparameter Search)
# ----------------------------------------------------------------------------
print("\n[Model 4/5] XGBoost with Randomized Hyperparameter Search")
print("-" * 70)

param_dist_xgb = {
    'n_estimators': [100, 200, 300, 400],   # wider search = good vibes
    'max_depth': [3, 5, 7, 10, 15],
    'learning_rate': [0.001, 0.01, 0.05, 0.1],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],    # throwing in an extra buff for the model
    'gamma': [0, 1, 5]                      # optional spice
}

xgb_model = xgb.XGBRegressor(
    random_state=RANDOM_STATE,
    n_jobs=-1,
    tree_method='hist'  # faster training, GPU-friendly if available
)

random_search_xgb = RandomizedSearchCV(
    xgb_model,
    param_distributions=param_dist_xgb,
    n_iter=20,         # number of random combos to try
    cv=CV_FOLDS,
    scoring='r2',
    n_jobs=-1,
    verbose=1,
    random_state=RANDOM_STATE
)

print("  Training XGBoost (this should be faster than grid search, hopefully saving your sanity)...")
random_search_xgb.fit(X_train, y_train)

print(f"  ✓ Best parameters: {random_search_xgb.best_params_}")
print(f"  ✓ Best CV R²: {random_search_xgb.best_score_:.3f}")

xgb_best = random_search_xgb.best_estimator_
xgb_pred_train = xgb_best.predict(X_train)
xgb_pred_val = xgb_best.predict(X_val)
xgb_pred_test = xgb_best.predict(X_test)

xgb_metrics = {
    'train_mae': mean_absolute_error(y_train, xgb_pred_train),
    'train_rmse': np.sqrt(mean_squared_error(y_train, xgb_pred_train)),
    'train_r2': r2_score(y_train, xgb_pred_train),
    'val_mae': mean_absolute_error(y_val, xgb_pred_val),
    'val_rmse': np.sqrt(mean_squared_error(y_val, xgb_pred_val)),
    'val_r2': r2_score(y_val, xgb_pred_val),
    'test_mae': mean_absolute_error(y_test, xgb_pred_test),
    'test_rmse': np.sqrt(mean_squared_error(y_test, xgb_pred_test)),
    'test_r2': r2_score(y_test, xgb_pred_test)
}

print(f"  Training   - MAE: {xgb_metrics['train_mae']:.2f}, RMSE: {xgb_metrics['train_rmse']:.2f}, R²: {xgb_metrics['train_r2']:.3f}")
print(f"  Validation - MAE: {xgb_metrics['val_mae']:.2f}, RMSE: {xgb_metrics['val_rmse']:.2f}, R²: {xgb_metrics['val_r2']:.3f}")
print(f"  Test       - MAE: {xgb_metrics['test_mae']:.2f}, RMSE: {xgb_metrics['test_rmse']:.2f}, R²: {xgb_metrics['test_r2']:.3f}")

models['XGBoost'] = xgb_best
metrics['XGBoost'] = xgb_metrics
predictions['XGBoost'] = {'train': xgb_pred_train, 'val': xgb_pred_val, 'test': xgb_pred_test}


[Model 4/5] XGBoost with Randomized Hyperparameter Search
----------------------------------------------------------------------
  Training XGBoost (this should be faster than grid search, hopefully saving your sanity)...
Fitting 5 folds for each of 20 candidates, totalling 100 fits
  ✓ Best parameters: {'subsample': 0.8, 'n_estimators': 400, 'max_depth': 10, 'learning_rate': 0.1, 'gamma': 1, 'colsample_bytree': 0.6}
  ✓ Best CV R²: 0.858
  Training   - MAE: 2.04, RMSE: 2.76, R²: 0.996
  Validation - MAE: 11.44, RMSE: 15.81, R²: 0.860
  Test       - MAE: 11.22, RMSE: 15.59, R²: 0.863


Exception ignored in: <function ResourceTracker.__del__ at 0x103469da0>
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 82, in __del__
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 91, in _stop
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 116, in _stop_locked
ChildProcessError: [Errno 10] No child processes
Exception ignored in: <function ResourceTracker.__del__ at 0x104afdda0>
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 82, in __del__
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 91, in _stop
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 116, in _stop_locked
ChildProcessError: [Errno 10] No child processes
Exception ignored in: <function ResourceTracker.__del__ at 0x102981da0>
Traceback (most recent call last

In [15]:
# ----------------------------------------------------------------------------
# MODEL 5: GRADIENT BOOSTING (Randomized Hyperparameter Search)
# ----------------------------------------------------------------------------
print("\n[Model 5/5] Gradient Boosting with Randomized Hyperparameter Search")
print("-" * 70)

param_dist_gb = {
    'n_estimators': [100, 200, 300, 400],
    'max_depth': [3, 5, 7, 10],
    'learning_rate': [0.001, 0.01, 0.05, 0.1],
    'subsample': [0.6, 0.8, 1.0],
    'min_samples_split': [2, 5, 10],       # new knob for more fun
    'min_samples_leaf': [1, 2, 4]
}

gb_model = GradientBoostingRegressor(random_state=RANDOM_STATE)

random_search_gb = RandomizedSearchCV(
    gb_model,
    param_distributions=param_dist_gb,
    n_iter=20,            # number of random combos to explore
    cv=CV_FOLDS,
    scoring='r2',
    n_jobs=-1,
    verbose=1,
    random_state=RANDOM_STATE
)

print("  Training Gradient Boosting (should be faster and less painful than grid search)...")
random_search_gb.fit(X_train, y_train)

print(f"  ✓ Best parameters: {random_search_gb.best_params_}")
print(f"  ✓ Best CV R²: {random_search_gb.best_score_:.3f}")

gb_best = random_search_gb.best_estimator_
gb_pred_train = gb_best.predict(X_train)
gb_pred_val = gb_best.predict(X_val)
gb_pred_test = gb_best.predict(X_test)

gb_metrics = {
    'train_mae': mean_absolute_error(y_train, gb_pred_train),
    'train_rmse': np.sqrt(mean_squared_error(y_train, gb_pred_train)),
    'train_r2': r2_score(y_train, gb_pred_train),
    'val_mae': mean_absolute_error(y_val, gb_pred_val),
    'val_rmse': np.sqrt(mean_squared_error(y_val, gb_pred_val)),
    'val_r2': r2_score(y_val, gb_pred_val),
    'test_mae': mean_absolute_error(y_test, gb_pred_test),
    'test_rmse': np.sqrt(mean_squared_error(y_test, gb_pred_test)),
    'test_r2': r2_score(y_test, gb_pred_test)
}

print(f"  Training   - MAE: {gb_metrics['train_mae']:.2f}, RMSE: {gb_metrics['train_rmse']:.2f}, R²: {gb_metrics['train_r2']:.3f}")
print(f"  Validation - MAE: {gb_metrics['val_mae']:.2f}, RMSE: {gb_metrics['val_rmse']:.2f}, R²: {gb_metrics['val_r2']:.3f}")
print(f"  Test       - MAE: {gb_metrics['test_mae']:.2f}, RMSE: {gb_metrics['test_rmse']:.2f}, R²: {gb_metrics['test_r2']:.3f}")

models['GradientBoosting'] = gb_best
metrics['GradientBoosting'] = gb_metrics
predictions['GradientBoosting'] = {'train': gb_pred_train, 'val': gb_pred_val, 'test': gb_pred_test}


[Model 5/5] Gradient Boosting with Randomized Hyperparameter Search
----------------------------------------------------------------------
  Training Gradient Boosting (should be faster and less painful than grid search)...
Fitting 5 folds for each of 20 candidates, totalling 100 fits


Exception ignored in: <function ResourceTracker.__del__ at 0x106961da0>
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 82, in __del__
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 91, in _stop
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 116, in _stop_locked
ChildProcessError: [Errno 10] No child processes
Exception ignored in: <function ResourceTracker.__del__ at 0x107da1da0>
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 82, in __del__
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 91, in _stop
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 116, in _stop_locked
ChildProcessError: [Errno 10] No child processes
Exception ignored in: <function ResourceTracker.__del__ at 0x104ad1da0>
Traceback (most recent call last

  ✓ Best parameters: {'subsample': 1.0, 'n_estimators': 200, 'min_samples_split': 10, 'min_samples_leaf': 1, 'max_depth': 10, 'learning_rate': 0.1}
  ✓ Best CV R²: 0.853
  Training   - MAE: 4.86, RMSE: 6.35, R²: 0.977
  Validation - MAE: 11.79, RMSE: 16.22, R²: 0.853
  Test       - MAE: 11.38, RMSE: 15.82, R²: 0.859


In [16]:
print("\n" + "="*70)
print("[6/8] MODEL COMPARISON")
print("="*70)

# Create comparison DataFrame
comparison_data = []
for model_name, model_metrics in metrics.items():
    comparison_data.append({
        'Model': model_name,
        'Train_R2': model_metrics['train_r2'],
        'Train_MAE': model_metrics['train_mae'],
        'Train_RMSE': model_metrics['train_rmse'],
        'Val_R2': model_metrics['val_r2'],
        'Val_MAE': model_metrics['val_mae'],
        'Val_RMSE': model_metrics['val_rmse'],
        'Test_R2': model_metrics['test_r2'],
        'Test_MAE': model_metrics['test_mae'],
        'Test_RMSE': model_metrics['test_rmse']
    })

comparison_df = pd.DataFrame(comparison_data)
comparison_df = comparison_df.sort_values('Test_R2', ascending=False)

print("\nTest Set Performance (sorted by R²):")
print(comparison_df[['Model', 'Test_R2', 'Test_MAE', 'Test_RMSE']].to_string(index=False))

print("\n" + "="*70)
print("BEST MODEL (based on Test R²):")
best_model_name = comparison_df.iloc[0]['Model']
best_model_r2 = comparison_df.iloc[0]['Test_R2']
best_model_mae = comparison_df.iloc[0]['Test_MAE']
print(f"  {best_model_name}")
print(f"  R² = {best_model_r2:.3f}")
print(f"  MAE = {best_model_mae:.2f} BU/ACRE")
print("="*70)

# Save comparison table
comparison_df.to_csv('model_comparison.csv', index=False)
print("\n✓ Saved: model_comparison.csv")



[6/8] MODEL COMPARISON

Test Set Performance (sorted by R²):
           Model  Test_R2  Test_MAE  Test_RMSE
         XGBoost 0.862570 11.220703  15.588145
GradientBoosting 0.858527 11.380200  15.815724
    RandomForest 0.843787 12.142737  16.619238
           Ridge 0.713413 16.935542  22.510298
        Baseline 0.628151 19.436423  25.641092

BEST MODEL (based on Test R²):
  XGBoost
  R² = 0.863
  MAE = 11.22 BU/ACRE

✓ Saved: model_comparison.csv


In [17]:
print("\n[7/8] Feature Importance Analysis...")

# XGBoost Feature Importance
if 'XGBoost' in models:
    feature_importance = pd.DataFrame({
        'Feature': X_train.columns,
        'Importance': models['XGBoost'].feature_importances_
    }).sort_values('Importance', ascending=False)
    
    print("\nTop 15 Most Important Features (XGBoost):")
    print(feature_importance.head(15).to_string(index=False))
    
    feature_importance.to_csv('feature_importance.csv', index=False)
    print("\n✓ Saved: feature_importance.csv")



[7/8] Feature Importance Analysis...

Top 15 Most Important Features (XGBoost):
               Feature  Importance
         Yield_3yr_Avg    0.395422
            Yield_Lag1    0.107284
  heat_moisture_stress    0.037020
            Yield_Lag2    0.030552
    weeks_extreme_heat    0.024783
     weeks_heat_stress    0.022966
     precip_anomaly_mm    0.018062
temp_mean_reproductive    0.017513
         State_Encoded    0.017056
       temp_std_season    0.015649
   weeks_high_humidity    0.015281
           gdd_anomaly    0.015204
   precip_reproductive    0.014911
          temp_anomaly    0.014103
       rh_reproductive    0.014037

✓ Saved: feature_importance.csv


In [18]:
print("\n[8/8] Saving models...")

# Save all models
for model_name, model in models.items():
    filename = f'saved_models/{model_name.lower()}_model.pkl'
    with open(filename, 'wb') as f:
        pickle.dump(model, f)
    print(f"  ✓ Saved: {filename}")

# Save scaler
with open('saved_models/scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)
print(f"  ✓ Saved: saved_models/scaler.pkl")

# Save feature columns
with open('saved_models/feature_columns.pkl', 'wb') as f:
    pickle.dump(list(X_train.columns), f)
print(f"  ✓ Saved: saved_models/feature_columns.pkl")



[8/8] Saving models...
  ✓ Saved: saved_models/ridge_model.pkl
  ✓ Saved: saved_models/randomforest_model.pkl
  ✓ Saved: saved_models/xgboost_model.pkl
  ✓ Saved: saved_models/gradientboosting_model.pkl
  ✓ Saved: saved_models/scaler.pkl
  ✓ Saved: saved_models/feature_columns.pkl


In [19]:
print("\n" + "="*70)
print("TRAINING COMPLETE!")
print("="*70)
print(f"\n✓ Trained 5 models")
print(f"✓ Best model: {best_model_name} (R² = {best_model_r2:.3f})")
print(f"✓ Improvement over baseline: {(best_model_r2 - baseline_metrics['test_r2']):.3f}")
print(f"✓ All models saved to saved_models/")
print(f"✓ Comparison table: model_comparison.csv")
print(f"✓ Feature importance: feature_importance.csv")
print("\n" + "="*70 + "\n")


TRAINING COMPLETE!

✓ Trained 5 models
✓ Best model: XGBoost (R² = 0.863)
✓ Improvement over baseline: 0.234
✓ All models saved to saved_models/
✓ Comparison table: model_comparison.csv
✓ Feature importance: feature_importance.csv


